## colab settings

In [ ]:
import os

prj_name = 'cifar-10'
prj_path = '/content/drive/My Drive/colab/study/image_classification/'\
        + prj_name + '/'
os.chdir(prj_path + 'notebooks/')

## settings

In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('..')

# main

In [ ]:
from importlib import import_module

import torch
import torch.nn as nn

#!pip install pytorch-ignite
#from ignite.metrics import Accuracy, Loss

#### VGG

In [ ]:
# model settings
model_mpath = 'src.models.vgg'
model_name = 'VGG'
model_cfg = {'cfg': [[64], [128], [256, 256], [512, 512], [512, 512]],
        'batch_norm': True}
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

#### InceptionV1

In [ ]:
# model settings
model_mpath = 'src.models.inception_v1'
model_name = 'InceptionV1'
model_cfg = [
        [{1: (0, 64), 3: (96, 128), 5: (16, 32), 'm': 32}, [False, 0, 0], None],
        [{1: (0, 128), 3: (128, 192), 5: (32, 96), 'm': 64}, [True, 3, 2], None],
        [{1: (0, 192), 3: (96, 208), 5: (16, 48), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 160), 3: (112, 224), 5: (24, 64), 'm': 64}, [False, 0, 0], 'aux'],
        [{1: (0, 128), 3: (128, 256), 5: (24, 64), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 112), 3: (144, 288), 5: (32, 64), 'm': 64}, [False, 0, 0], None],
        [{1: (0, 256), 3: (160, 320), 5: (32, 128), 'm': 128}, [True, 2, 2], 'aux'],
        [{1: (0, 256), 3: (160, 320), 5: (32, 128), 'm': 128}, [False, 0, 0], None],
        [{1: (0, 384), 3: (192, 384), 5: (48, 128), 'm': 128}, [False, 0, 0], 'final']
        ]
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
class loss_cls(nn.Module):
    def __init__(self):
        super(loss_cls, self).__init__()
    def forward(self, inp, tar):
        loss = 0
        loss_fn = nn.CrossEntropyLoss()
        inps = inp.split(10, 1)
        for i, p in enumerate(inps):
            if i < len(inps) - 1:
                loss += 0.3 * loss_fn(p, tar)
            else:
                loss += 1.0 * loss_fn(p, tar)
        return loss
loss_fn = loss_cls()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

#### ResNet

In [ ]:
# model settings
model_mpath = 'src.models.resnet'
model_name = 'ResNet'
model_cfg = [
        [(3, 64), (3, 64)],
        [(3, 64), (3, 64)],
        [(3, 128), (3, 128)],
        [(3, 128), (3, 128)],
        [(3, 256), (3, 256)],
        [(3, 256), (3, 256)],
        [(3, 512), (3, 512)],
        [(3, 512), (3, 512)]
        ]
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000
# startblock batchnorm eps check

#### DenseNet

In [ ]:
# model settings
model_mpath = 'src.models.densenet'
model_name = 'DenseNet'
gr = 4
cr = 0.5
model_cfg = {
        'start': 2 * gr,
        'dense': [
                [(4 * gr, gr)] * 4,
                [(4 * gr, gr)] * 8,
                [(4 * gr, gr)] * 12,
                #[(4 * gr, gr)] * 16
                ],
        'compress': cr
        }
init_weights = True

# dataset, dataloader settings
batch_size = 2500

# train settings
loss_fn = nn.CrossEntropyLoss()
opt_ = torch.optim.Adam
lr = 0.00003
#val_metrics = {'acc': Accuracy(), 'loss': Loss(loss_fn)}
device = 'cuda:0'
max_epochs = 1000

## model construction

In [ ]:
model_cls = getattr(
        import_module(model_mpath),
        model_name
        )
model = model_cls(model_cfg, init_weights=init_weights)
for b in model.named_children():
    print(b)

('B_000', DNStartBlock(
  (B_000): Conv2d(3, 8, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (B_001): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (B_002): ReLU(inplace=True)
  (B_003): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('B_001', DNDenseBlock(
  (B_000): DNCompositeBlock(
    (B_000): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_001): ReLU(inplace=True)
    (B_002): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (B_003): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_004): ReLU(inplace=True)
    (B_005): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (B_001): DNCompositeBlock(
    (B_000): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (B_001): ReLU(inplace=True)
    (B_002): Conv2d(12, 16, kernel_size=(1, 1), stride=(

## load data

In [ ]:
dpath = '../data/raw/'

train_folds = []
for i in range(1, 6):
    train_folds.append(torch.load(dpath + 'data_batch_' + str(i) + '.pt'))
test_set = torch.load(dpath + 'test_batch.pt')

In [ ]:
from collections import Counter

for train_fold in train_folds:
    print(sorted(Counter(s['label'] for s in train_fold).items()))
print(sorted(Counter(s['label'] for s in test_set).items()))

[(0, 1005), (1, 974), (2, 1032), (3, 1016), (4, 999), (5, 937), (6, 1030), (7, 1001), (8, 1025), (9, 981)]
[(0, 984), (1, 1007), (2, 1010), (3, 995), (4, 1010), (5, 988), (6, 1008), (7, 1026), (8, 987), (9, 985)]
[(0, 994), (1, 1042), (2, 965), (3, 997), (4, 990), (5, 1029), (6, 978), (7, 1015), (8, 961), (9, 1029)]
[(0, 1003), (1, 963), (2, 1041), (3, 976), (4, 1004), (5, 1021), (6, 1004), (7, 981), (8, 1024), (9, 983)]
[(0, 1014), (1, 1014), (2, 952), (3, 1016), (4, 997), (5, 1025), (6, 980), (7, 977), (8, 1003), (9, 1022)]
[(0, 1000), (1, 1000), (2, 1000), (3, 1000), (4, 1000), (5, 1000), (6, 1000), (7, 1000), (8, 1000), (9, 1000)]


In [ ]:
from torch.utils.data import ConcatDataset, DataLoader

train_set = ConcatDataset([f for j, f in enumerate(train_folds) if j != 4])
val_set = train_folds[4]

class SkorchDataset(torch.utils.data.Dataset):
    def __init__(self, ds):
        self.ds = ds

    def __getitem__(self, idx):
        return tuple(self.ds[idx].values())

    def __len__(self):
        return len(self.ds)

train_set = SkorchDataset(train_set)
val_set = SkorchDataset(val_set)

train_loader = DataLoader(train_set, batch_size=batch_size,
        shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size,
        shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size,
        shuffle=True, num_workers=2)

del train_folds

## train

In [ ]:
train_set[0][0]

tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.0314,  ..., 0.1961, 0.1961, 0.

In [ ]:
%%capture
!pip install skorch

from skorch.net import NeuralNet
from skorch.helper import predefined_split
from skorch.callbacks import EpochScoring

from sklearn.utils.multiclass import type_of_target
import numpy as np
from sklearn.metrics import accuracy_score
def argmax_acc(skorch_model, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = skorch_model.predict(ds).argmax(1)
    return accuracy_score(y_true, y_pred)
train_acc_cb = EpochScoring(argmax_acc, lower_is_better=False,
        on_train=True, name='train_acc') # history에서 불러오는 뭐시기가 있대, 10epoch마다 보이기?
val_acc_cb = EpochScoring(argmax_acc, lower_is_better=False,
        name='valid_acc')

skorch_model = NeuralNet(
        model,
        nn.CrossEntropyLoss,
        opt_,
        lr,
        max_epochs,
        batch_size,
        train_split=predefined_split(val_set),
        callbacks=[train_acc_cb, val_acc_cb],
        device=device
        )

In [ ]:
skorch_model.fit(train_set)

  epoch    train_acc    train_loss    val_acc    valid_loss     dur
-------  -----------  ------------  ---------  ------------  ------
      1       0.2425        2.0852     0.2340        2.0932  3.0215
      2       0.2453        2.0763     0.2373        2.0851  2.7635
      3       0.2479        2.0677     0.2413        2.0770  2.7546
      4       0.2511        2.0591     0.2453        2.0689  2.8692
      5       0.2548        2.0507     0.2488        2.0613  2.7533
      6       0.2576        2.0425     0.2510        2.0533  2.7470
      7       0.2611        2.0344     0.2525        2.0458  2.7601
      8       0.2635        2.0265     0.2544        2.0385  2.7429
      9       0.2664        2.0187     0.2581        2.0308  2.8740
     10       0.2692        2.0111     0.2615        2.0235  2.7271
     11       0.2722        2.0036     0.2646        2.0162  2.7587
     12       0.2759        1.9961     0.2694        2.0086  2.7312
     13       0.2789        1.9889     0.2713   

<class 'skorch.net.NeuralNet'>[initialized](
  module_=DenseNet(
    (B_000): DNStartBlock(
      (B_000): Conv2d(3, 8, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (B_001): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (B_002): ReLU(inplace=True)
      (B_003): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (B_001): DNDenseBlock(
      (B_000): DNCompositeBlock(
        (B_000): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (B_001): ReLU(inplace=True)
        (B_002): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (B_003): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (B_004): ReLU(inplace=True)
        (B_005): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (B_001): DNCompositeBlock(
        (B_000): BatchNorm2d(12, eps=1e-05, momentum=0.

In [ ]:
from src.models.train_model import train_net

opt = opt_(model.parameters(), lr)

trainer = train_net(model, opt, loss_fn, val_metrics,
        train_loader, val_loader, device)
trainer.run(train_loader, max_epochs=max_epochs)

Epoch 1
Train - acc: 0.11 loss: 2.33 
Val   - acc: 0.11 loss: 2.33 
Epoch 2
Train - acc: 0.12 loss: 2.32 
Val   - acc: 0.12 loss: 2.32 
Epoch 3
Train - acc: 0.13 loss: 2.31 
Val   - acc: 0.13 loss: 2.31 
Epoch 4
Train - acc: 0.14 loss: 2.30 
Val   - acc: 0.14 loss: 2.29 
Epoch 5
Train - acc: 0.14 loss: 2.28 
Val   - acc: 0.15 loss: 2.28 
Epoch 6
Train - acc: 0.15 loss: 2.27 
Val   - acc: 0.15 loss: 2.27 
Epoch 7
Train - acc: 0.15 loss: 2.26 
Val   - acc: 0.16 loss: 2.25 
Epoch 8
Train - acc: 0.16 loss: 2.24 
Val   - acc: 0.17 loss: 2.24 
Epoch 9
Train - acc: 0.17 loss: 2.23 
Val   - acc: 0.17 loss: 2.23 
Epoch 10
Train - acc: 0.17 loss: 2.22 
Val   - acc: 0.17 loss: 2.22 
Epoch 11
Train - acc: 0.18 loss: 2.21 
Val   - acc: 0.18 loss: 2.21 
Epoch 12
Train - acc: 0.18 loss: 2.20 
Val   - acc: 0.19 loss: 2.20 
Epoch 13
Train - acc: 0.19 loss: 2.18 
Val   - acc: 0.19 loss: 2.19 
Epoch 14
Train - acc: 0.19 loss: 2.17 
Val   - acc: 0.20 loss: 2.17 
Epoch 15
Train - acc: 0.20 loss: 2.16 
Val 

Engine run is terminating due to exception: .
Engine run is terminating due to exception: .
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/li

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-f6d46458769a>", line 7, in <module>
    trainer.run(train_loader, max_epochs=max_epochs)
  File "/usr/local/lib/python3.6/dist-packages/ignite/engine/engine.py", line 658, in run
    return self._internal_run()
  File "/usr/local/lib/python3.6/dist-packages/ignite/engine/engine.py", line 722, in _internal_run
    self._handle_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/ignite/engine/engine.py", line 437, in _handle_exception
    raise e
  File "/usr/local/lib/python3.6/dist-packages/ignite/engine/engine.py", line 710, in _internal_run
    self._fire_event(Events.EPOCH_COMPLETED)
  File "/usr/local/lib/python3.6/dist-packages/ignite/engine/engine.py", line 393, in _fire_event
    func(*first, *(event_args + others), **kwargs)
  File "../src/models/train_m

KeyboardInterrupt: ignored

## Test